# Download representative images of volcanos


Credit: this script derived from [Aaron Geller's blog post.](https://sites.northwestern.edu/researchcomputing/2021/11/19/downloading-satellite-images-made-easy/), including the use of functions based on his [EarthEngineToGeoTIFF repository.](https://github.com/ageller/EarthEngineToGeoTIFF)

In [ ]:
import ee
%pip install rasterio

import rasterio
from rasterio.plot import show as showRasterio

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta
import zipfile
import os
import requests

import numpy as np
import time

In [2]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
OUTPUT_IMAGE_DIR = PROJECT_DIR +"Full_Sentinel_1_10km/"
PRIOR_OUTPUT_IMAGE_DIR = PROJECT_DIR +"Full_sentinel_1_10km_prior_year/"

TIFS_DIR = DRIVE_MOUNT+"/My Drive/tifs/"

targets = pd.read_excel(PROJECT_DIR + "Sample Eruptions.xlsx")


Mounted at /content/drive


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
sleep_interval = 20
wait_limit = 600

download = True    

def get_sentinel_1_image(lon, lat, sze_m, filename_base, 
                         dateMin = '2020-04-01', dateMax = '2020-04-30', 
                         output_dir = OUTPUT_IMAGE_DIR):
    '''    
    download an SARimage from the Sentinal S2 Surface Reflectance satellite, at the given coordinates
    
    lon : central longitude in degrees
    lat : central latitude in degrees
    sze_km : size of the edge of the box in km
    dateMin : minimum date to use for image search in year-month-day (e.g., 2020-08-01)
    dateMax : maximum date to use for image search in year-month-day (e.g., 2020-08-31)
    base_filename : base output filename for the GeoTIFF images. Will create 6 files for each bse
     For each of ASCENDING and DECENDING:
      - mean of VH channel
      - mean of VV channel
     mean of Ascending and descending:
      - VV channel
      - VH channel 
    
    https://developers.google.com/earth-engine/guides/sentinel1
    '''

    print(f"{lon} {lat} {filename_base} {dateMin} {dateMax} {output_dir}")
    if (download):
      # Load the Sentinel-1 ImageCollection, filter to Jun-Sep 2020 observations.
      sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
                          .filterDate(dateMin, dateMax)

      # Calculate Area of Interest
      pt = ee.Geometry.Point(lon,lat)
          
      aoi = pt.buffer(distance=sze_m)

      # Filter the Sentinel-1 collection by metadata properties.
      vvVhIw = sentinel1\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
        .filter(ee.Filter.eq('instrumentMode', 'IW'));

    
      #Separate ascending and descending orbit images into distinct collections.
      asc = vvVhIw.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
                .mean().addBands(ee.Image.pixelLonLat())
      desc = vvVhIw.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
                .mean().addBands(ee.Image.pixelLonLat())

      task_list = []
      col_names = ["a", "d"]
      for idb, db in enumerate([asc, desc]):
        for selection in ['VV','VH']:
          #print(f"Task: {filename_base} {col_names[idb]}{selection}")
          task = ee.batch.Export.image.toDrive(image=db.select(selection),
                                            description=filename_base+"_"+col_names[idb]+selection,
                                            scale=10,
                                            region=aoi,
                                            fileNamePrefix=filename_base+"_"+col_names[idb]+selection,
                                            folder="tifs",
                                            crs='EPSG:4326',
                                            fileFormat='GeoTIFF')

          task.start()
          task_list.append(task)
      
      loop_count= 0

      for task in task_list:
        while(task.active() and sleep_interval*loop_count < wait_limit):
           loop_count += 1
           time.sleep(sleep_interval)
           #print(f'slept {sleep_interval*loop_count}')
          
        status = task.status()

        if (status['state'] == 'COMPLETED' or status['state'] == 'READY'):
            print(f"Success: {task}")
        else:
            print (status['state'])
            if (status['state'] == "FAILED"):
               print(f"Failed: {task} {status['error_message']}")

    time.sleep(30) # sleep for final file to write

    # merge channel tifs into one image

    for ilayer, layer in enumerate(["aVH","aVV","dVH","dVV"]):
    # convert image to png
       f,ax = plt.subplots(figsize=(15,15))
       input = rasterio.open(TIFS_DIR+filename_base+"_"+layer+".tif")
       showRasterio(input.read(), ax = ax, transform=input.transform)  
       
       output = output_dir+filename_base+"_"+layer+".png"
       f.savefig(output, bbox_inches='tight')
       input.close()
       plt.close(f)

       print(f"wrote {output}")
    
    
    # clean up tifs left behind
    lst = os.listdir(TIFS_DIR)
    regex_pattern = filename_base + ".*.tif"
    for f in lst:
       if re.match(regex_pattern, f):
         os.remove(TIFS_DIR+"/"+f)

In [ ]:
target_size_m = 10000
pull_eruption_year = True
pull_prior_year = True
for i_eruption in range(25,targets.shape[0]):
    volc = targets.loc[i_eruption]
    v_name = re.sub(r"[,.]","",re.sub(r"\s+", "_",volc["Volcano Name"]))
   

    # adjust eruption date by 1 year
    erupt_start_date = date(volc["Start Year"].astype(int),
                            volc["Start Month"].astype(int),
                            volc["Start Day"].astype(int))
    erupt_end_date = date(volc["End Year"].astype(int),
                          volc["End Month"].astype(int),
                          volc["End Day"].astype(int))

    if (erupt_start_date >= date(2015,6,23)):
      print(v_name,erupt_start_date)
      if pull_eruption_year:
        # do sometime before the eruption too
        pre_erupt_date = erupt_start_date - timedelta(weeks=6)

        volc_file_name = f'{v_name}_{pre_erupt_date}_pre_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
        print(i_eruption, volc_file_name)
          
        try:
            _ =get_sentinel_1_image(volc["Longitude"], volc["Latitude"], target_size_m,
                               volc_file_name,
                               output_dir = OUTPUT_IMAGE_DIR,
                               dateMin = pre_erupt_date.isoformat(), dateMax=erupt_start_date.isoformat())
        except Exception as e:
            print(f'Failed to get data for {volc_file_name}')
            print(e)
      
      if pull_prior_year:
        # do prior to eruption too
        prior_pre_erupt_date = erupt_start_date - timedelta(weeks=6)-timedelta(weeks=52)
        prior_erupt_start_date = erupt_start_date -timedelta(weeks=52)
        prior_volc_file_name = f'{v_name}_{prior_pre_erupt_date}_pre_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
        print(i_eruption, volc_file_name)
          
        try:
            _ =get_sentinel_1_image(volc["Longitude"], volc["Latitude"], target_size_m,
                               volc_file_name,
                               output_dir = PRIOR_OUTPUT_IMAGE_DIR,
                               dateMin =  prior_pre_erupt_date.isoformat(), dateMax=prior_erupt_start_date.isoformat())
        except Exception as e:
            print(f'Failed to get data for {volc_file_name}')
            print(e)
      next_start_date = erupt_start_date
      next_end_date = erupt_start_date + timedelta(days=7)
      while (next_end_date <= erupt_end_date + timedelta(days=7) and next_end_date <= erupt_start_date + timedelta(days=97)):
          volc_file_name = f'{v_name}_{next_start_date}_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'
          prior_volc_file_name = f'{v_name}_{next_start_date-timedelta(weeks=52)}_{volc["Volcano Number"]:.0f}_{volc["Eruption Number"]:.0f}'

          print(i_eruption, volc_file_name)
          if pull_eruption_year:
            try:
                _ =get_sentinel_1_image(volc["Longitude"], volc["Latitude"], 
                            target_size_m, volc_file_name,
                            output_dir = OUTPUT_IMAGE_DIR,
                            dateMin = next_start_date.isoformat(), 
                            dateMax=next_end_date.isoformat())
            except Exception as e:
              print(f'Failed to get data for {volc_file_name}:\n {e}')
          if pull_prior_year:
            try:
                _ =get_sentinel_1_image(volc["Longitude"], volc["Latitude"], 
                            target_size_m, prior_volc_file_name,
                            output_dir = PRIOR_OUTPUT_IMAGE_DIR,
                            dateMin = (next_start_date-timedelta(weeks=52)).isoformat(), 
                            dateMax=(next_end_date-timedelta(weeks=52)).isoformat())
            except Exception as e:
              print(f'Failed to get prior year data for {prior_volc_file_name}:\n {e}')
          next_start_date = next_start_date + timedelta(days=7)
          next_end_date = next_end_date + timedelta(days=7)
        

In [ ]:
lat = 12.702;
lon = -87.004;
dateMin = '2020-12-27';
dateMax = '2021-1-01';
sze_m = 1000;
filename_base = "San_Cristobal"

filename_base = "Kuchinoerabujima"
lon = 130.217
lat = 30.443

_ =get_sentinel_1_image(lon,lat,sze_m,filename_base,dateMin,dateMax)